# Life cycle assessment

For a sustainable bioeconomy, it is critical to assess the environmental impact of a production process. This chapter goes over BioSTEAM's life cycle assessment (LCA) capabilities through a cradle-to-biorefinery-gate LCA of a sugarcane biorefinery that produces electricity and ethanol. We focus on the global warning potential (GWP) because it is central to policy implementation.

## Example

A biorefinery system defines the life cycle inventory. Apart from the model itself, only the characterization factors for inputs and outputs are needed to perform LCA. These can be retrieved from standard public models like the Greenhouse Gases, Regulated Emissions, and Energy Use in Transportation (Argonne GREET Model). Here we load a sugarcane to ethanol biorefinery, define characterization factors from GREET, and perform LCA using energy based, revenue base, and displacement allocation methods (i.e. system expansion).

Define the impact indicator and the characterization factors:

In [ ]:
### Load system
import biosteam as bst
from warnings import filterwarnings; filterwarnings('ignore') 
from biorefineries import cane
sc = cane.Biorefinery('S1') # Sugarcane conventional ethanol
system = sc.system

### Set characterization factors on a per kg basis, as available in GREET 2020

# Key for GWP characterization factor; we make it informative
# but any value can work, even just a number.
# Note that the impact assessment methodology used in GREET is the
# Tool for the Reduction and Assessment of Chemical and other 
# Environmental Impacts (TRACI)
GWP = 'GWP 100yr' 
bst.settings.define_impact_indicator(key=GWP, units='kg*CO2e')

# Sugarcane for ethanol production, adjusted for moisture content (0.75 originally, 0.70 in the model).
sc.sugarcane.set_CF(GWP, 0.02931 * 0.30 / 0.25, basis='kg', units='kg*CO2e') 

# Production of phosphoric acid from P2O5
sc.H3PO4.set_CF(GWP, 1.) # Basis defaults to kg; units default to defined units

# NG-Fired Simple-Cycle Gas Turbine CHP Plant, no transmission included
bst.settings.set_electricity_CF(GWP, 0.36, basis='kWhr', units='kg*CO2e')

# Lime production from lime stone, adjusted for dilution
lime_dilution = 1 - sc.lime.get_mass_composition('Water')
sc.lime.set_CF(GWP, 1.28 * lime_dilution)

# Gasoline blendstock from Crude Oil for US Refineries
sc.denaturant.set_CF(GWP, 0.84)

# Assume all other feeds are negligible.

Compute the GWP per kg of ethanol using displacement and energy allocation:

In [ ]:
# Displacement allocation [kg-CO2e / kg-ethanol]
GWP_sugarcane_ethanol_displacement = system.get_net_impact(key=GWP) / system.get_mass_flow(sc.ethanol) 

# Energy allocation by gasoline gallon equivalent (GGE)
GWP_per_GGE = system.get_property_allocated_impact(
    key=GWP, name='energy', basis='GGE', # Energy basis defaults to 'kJ'
    ignored={sc.yeast, sc.filter_cake}
) # kg-CO2e / GGE

GWP_sugarcane_ethanol_energy = (
    GWP_per_GGE * sc.ethanol.get_property('LHV', 'GGE/hr') / sc.ethanol.F_mass
) # kg-CO2e / kg sugarcane ethanol

print(
    "Cradle to biorefinery gate GWP of sugarcane ethanol:\n "
    f"Displacement allocation: {GWP_sugarcane_ethanol_displacement:.2f} [kg CO2e / kg]\n "
    f"Energy allocation: {GWP_sugarcane_ethanol_energy:.2f} [kg CO2e / kg]\n "
)

Note that biogenic emissions do not contribute any GWP while that non-biogenic emissions associated to the bioreinfery do. In this example, all emissions are biogenic. The GWP computed here using energy allocation is very close to the value available in the [Ecoinvent life cycle inventory](https://ecoinvent.org/) (2020 database) for a modern autonomous sugarcane ethanol plant in Brazil (value not shown here to avoid proprietary issues).

## Life cycle inventory and impact breakdown

A breakdown of the inventory and impacts comes in handy for inpecting values and verifying results. The `biosteam.report` module includes a few helpful functions for this:

In [ ]:
from biosteam import report
report.lca_inventory_table(
    systems=[system], 
    key=GWP,
    items=[sc.ethanol], # For including products without characterization factors
)

In [ ]:
report.lca_displacement_allocation_table(
    systems=[system], 
    key=GWP,
    items=[sc.ethanol], # For dividing yearly impact by ethanol production  
)

In [ ]:
report.lca_property_allocation_factor_table(
    systems=[system],
    property='energy',
)

In [ ]:
report.lca_displacement_allocation_factor_table(
    systems=[system],
    items=[sc.ethanol],
    key=GWP,
)

## Problem
Compute the GWP per kg of ethanol using revenue allocation:

In [ ]:
# Economic/revenue allocation
GWP_per_USD = system.get_property_allocated_impact(
    key=None, name=None, basis=None, # Revenue basis defaults to USD
) # kg-CO2e / USD
GWP_sugarcane_ethanol_revenue = (
    GWP_per_USD * None
) # kg-CO2e / kg-ethanol

print(
    f"Economic allocation: {GWP_sugarcane_ethanol_revenue:.2f} [kg CO2e / kg]"
)

Compute revenue allocation factors:

In [ ]:
allocation_factors = report.lca_property_allocation_factor_table(
    systems=[system],
    property=None,
)
allocation_factors

Use allocation factors to compute the carbon intensity:

In [ ]:
allocation_factor = float(allocation_factors.loc['Advanced ethanol'])
GWP_sugarcane_ethanol_revenue = None * system.get_total_feeds_impact(GWP) / system.get_mass_flow(sc.ethanol) 
print(
    f"Economic allocation: {GWP_sugarcane_ethanol_revenue:.2f} [kg CO2e / kg]"
)